<a href="https://colab.research.google.com/github/fetichex/github-final-project/blob/main/text_classification_mbert_robust_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Entrenamiento de clasificación de texto con mBERT (Robusto y Completo)
Este notebook te guía a través del proceso de entrenar un modelo de clasificación de texto utilizando mBERT y tu conjunto de datos, considerando correcciones y precauciones para evitar posibles errores.



## Preparación del entorno
Instala las bibliotecas necesarias.


In [1]:

!pip install transformers[torch] accelerate datasets -U



## Cargar el conjunto de datos
Asegúrate de subir o tener disponibles los archivos CSV que contienen los conjuntos de entrenamiento, validación y prueba.


In [2]:

import pandas as pd
import numpy as np

train_data = pd.read_csv('train_data.csv')
valid_data = pd.read_csv('valid_data.csv')
test_data = pd.read_csv('test_data.csv')

# Mapeo de categorías a índices
categories = list(train_data['Categoría'].unique())
category2index = {category: index for index, category in enumerate(categories)}



## Cargar un modelo preentrenado y su tokenizador
Vamos a utilizar mBERT (BERT multilingüe) para este proceso.


In [3]:

from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(categories))


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



## Preprocesamiento de datos
Tokeniza las oraciones y prepáralas para el modelo.


In [4]:

train_encodings = tokenizer(list(train_data['Actividad']), truncation=True, padding=True, max_length=256)
valid_encodings = tokenizer(list(valid_data['Actividad']), truncation=True, padding=True, max_length=256)



## Preparar el conjunto de datos para PyTorch


In [5]:

import torch

class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = [category2index[label] for label in labels]

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = Dataset(train_encodings, list(train_data['Categoría']))
valid_dataset = Dataset(valid_encodings, list(valid_data['Categoría']))



## Métricas de evaluación
Definimos las métricas que queremos utilizar durante la evaluación.


In [6]:

from datasets import load_metric

accuracy_metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return accuracy_metric.compute(predictions=predictions, references=labels)


<ipython-input-6-ed26a4f047a4>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  accuracy_metric = load_metric("accuracy")



## Entrenar el modelo


In [7]:

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir='./results',
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics  # Añade esta línea
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,3.041600,2.997207,0.000000
2,3.003000,3.016917,0.050000
3,2.949100,2.965331,0.050000


TrainOutput(global_step=45, training_loss=2.997743754916721, metrics={'train_runtime': 48.4932, 'train_samples_per_second': 7.424, 'train_steps_per_second': 0.928, 'total_flos': 3515567330880.0, 'train_loss': 2.997743754916721, 'epoch': 3.0})


## Evaluar el modelo
Una vez que el modelo esté entrenado, evalúa su rendimiento en el conjunto de prueba.


In [8]:
# Guardar el modelo
model_path = "./ownBERTo"
model.save_pretrained(model_path)

# Guardar el tokenizador
tokenizer.save_pretrained(model_path)


('./ownBERTo/tokenizer_config.json',
 './ownBERTo/special_tokens_map.json',
 './ownBERTo/vocab.txt',
 './ownBERTo/added_tokens.json')